In [2]:
# !git clone https://github.com/rr2n17/AdvML.git

In [3]:
!ls

data		    toxic_french_2000.csv  upsampling.ipynb
obscene_french.csv  toxic_french.csv	   upsampling_old.ipynb
test.csv	    train.csv


## Install libraries

In [4]:
# !pip install --upgrade pip
# !pip install nltk
# !apt-get install libenchant1c2a
# !pip install pyenchant
# !pip install joblib textblob -U
# !pip install ipyparallel

In [5]:
import pandas as pd
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
from random import randint
import nltk.data
from nltk.tokenize import TreebankWordTokenizer
import re
import enchant
from enchant.checker import SpellChecker
from joblib import Parallel, delayed
from textblob import TextBlob
from textblob.translate import NotTranslated
from ipyparallel import Client
import time

In [6]:
# nltk.download("averaged_perceptron_tagger")
# nltk.download("wordnet")
# nltk.download('punkt')
# nltk.download('stopwords')

## Read data

In [7]:
path = "./train.csv"
dataset = pd.read_csv(path)
print(dataset.shape)
dataset.head()

(159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [76]:
c = Client()
c.ids[-3:]
cc = c[:]
cc50 = c[:50]
cc100 = c[:]

In [77]:
# @cc.remote(block=True)
# def getpid():
#     import os
#     return os.getpid()
# print(getpid())

## Upsample

In [78]:
# import time

# def timef(func):
#     t1 = time.time()
#     res = func
#     t2 = time.time()
#     print(t2 - t1)
#     return [res] 

# @cc.parallel(block=True)
# # @timef
# def test(lst):
#     l = []
#     for i in lst:
#         l.append(i)
#     return len(l)

# x = range(10000000)

# t1 = time.time()
# print(test(x))
# t2 = time.time()
# print(t2-t1)


In [119]:
toxic = dataset[dataset['identity_hate'] == 1]
print(toxic.shape)

(1405, 8)


In [120]:
%px import re
%px from enchant.checker import SpellChecker

# tokenizer = TreebankWordTokenizer()
chkr = SpellChecker("en_UK", "en_US")

@cc50.parallel(block=True)
def preprocess(list_of_comments):
    """
    
    Replaces words that are written mistakenly with the 
    closest (enchant.checker.Spellchecker) and turns them to the lowercase then
    
    External libraries used:
    :enchant.SpellChecker: not found words are replaced with the most similar
    
    IO:
    :type comment: str
    :rtype: str
    
    """
    res = []
    for comment in list_of_comments:
        ## retrieve alphanumeric and underscore 
        ## words, replace with " " if matches
        t = type(comment)
        comment = re.sub(r'[\W_]+', " ", str(comment)).lower()
        ## run through checker if the words exist
        ## replace words that are not found with a recommended word
        chkr = SpellChecker("en_UK", "en_US")
        chkr.set_text(comment)
        for err in chkr:
            if err.suggest():
                sug = err.suggest()[0] # take the first item from the recommended words
                err.replace(sug)
        res.append(chkr.get_text())
    return res

t1 = time.time()
toxic = toxic.reset_index(drop=True)
toxic['comment_text'] = pd.Series(preprocess(list(toxic['comment_text'])))
t2 = time.time()
print(t2 - t1)

25.99495577812195


In [121]:
toxic.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
1400,fb726deec64157bd,lo you re gay you will never know how good it ...,1,1,1,0,1,1
1401,fc3efa2f6f025f6d,oh fuck off the pansy Jew would just whine abo...,1,0,1,0,1,1
1402,fd052883fa6a8697,shalom Semite get the fuck out of here i will ...,1,1,1,1,1,1
1403,fdce660ddcd6d7ca,i think he is a gay fag,1,0,0,0,0,1
1404,fef4cf7ba0012866,our previous conversation you fucking shit ea...,1,0,1,0,1,1


In [122]:
%px from textblob import TextBlob
%px from textblob.translate import NotTranslated

@cc100.parallel(block=True)
def upsample_via_lang(comments, langs):
    """
    
    Upsamples by creating variations of a comment
    by translating from English -> German -> French -> English
    
    :type list_of_comments: list of commentaries of a string type
    :rtype: a new variation of commentaries
    
    """
    res = []
    for comment in comments:
        if not isinstance(comment, str):
            comment = str(comment)
        if hasattr(comment, "decode"):
            comment = comment.decode("utf-8")
        text = TextBlob(comment)
        try:
            text = text.translate(to='de')
            text = text.translate(to='fr')
            text = text.translate(to="en")
            text = re.sub(r'[\W_]+', ' ', str(text))
        except NotTranslated:
            pass
        res.append(text)
    return [re.sub(r'[\W_]+', ' ', str(text)) for text in res]

t1 = time.time()
comments = list(toxic['comment_text'])
lang = 'fr'
langs = [lang for i in range(len(comments))]
comments_from_french = pd.Series(upsample_via_lang(comments, langs))
t2 = time.time()
print(t2 - t1)
# x.head()


15.520253658294678


In [123]:
# Load the pretrained neural net
tokenizer = nltk.data.load('../../../nltk_data/tokenizers/punkt/english.pickle')

dp = pd.DataFrame()
dp["fr"] = comments_from_french

for text in dp["fr"]:
    text = str(text)
    output=""
    # Tokenize the text
    tokenized = tokenizer.tokenize(text)

    # Get the list of words from the entire text
    words = word_tokenize(text)

    # Identify the parts of speech
    tagged = nltk.pos_tag(words)

    for i in range(0,len(words)):
        
        replacements = []

        # Only replace nouns with nouns, vowels with vowels etc.
        for syn in wordnet.synsets(words[i]):

            # Do not attempt to replace proper nouns or determiners
            if tagged[i][1] == 'NNP' or tagged[i][1] == 'DT':
                break

            # The tokenizer returns strings like NNP, VBP etc
            # but the wordnet synonyms has tags like .n.
            # So we extract the first character from NNP ie n
            # then we check if the dictionary word has a .n. or not 
            word_type = tagged[i][1][0].lower()
            if syn.name().find("."+word_type+"."):
                # extract the word only
                r = syn.name()[0:syn.name().find(".")]
                replacements.append(r)

        if len(replacements) > 0:
            # Choose a random replacement
            replacement = replacements[randint(0,len(replacements)-1)]
            output = output + " " + replacement
        else:
            # If no replacement could be found, then just use the
            # original word
            output = output + " " + words[i]
    
    dp = dp.append(pd.Series({"fr":output}),ignore_index=True)

In [124]:
%px from nltk.corpus import wordnet, stopwords

@cc50.parallel(block=True)
def tok_and_rem(comments):
    res = []
    for x in comments:
        s = x.split()
        l = list(s)
        for word in s:
            if (len(word) <= 2):
                l.remove(word)
            elif word in stopwords.words('english'):
                l.remove(word)
        res.append(l)
    return res

In [125]:
print(dp.shape)
dp.tail()

(2810, 1)


,fr
2805,You embody brave you will never know how dece...
2806,ohio crap the Jew of Thought would have fair ...
2807,semitic shalom draw you out of here i will ki...
2808,one think helium s a gay fagot
2809,our previous conversation you fucking denounc...


In [126]:
t1 = time.time()
pd.Series(tok_and_rem(list(dp['fr'])))
t2 = time.time()
t2 - t1

8.915174722671509

In [130]:
dp.to_csv("identity_hate_french_2810.csv")

In [131]:
# toxic severe_toxic obscene threat insult identity_hate
dataset[dataset["insult"] == 1].count()

id               7877
comment_text     7877
toxic            7877
severe_toxic     7877
obscene          7877
threat           7877
insult           7877
identity_hate    7877
dtype: int64